# Extracting Metadata from Chapters with Outlines 

In [ ]:

import re
import numpy as np
import pandas as pd
from pprint import pprint
import json

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preparing the Data 

In [38]:
df_outlines = pd.read_csv("F:\Environmental Baseline Data\Version 3\Indices\Outline_Present_htmlTOC.csv")
 
len(df_outlines)
df_outlines[20:30].head(5)

1328

,Unnamed: 0,file_nos,file_names,file_paths,int_postals,TOC_present,consecutive_postals,chunk_nos,outline_present
20,20,1,A0Y1V1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62871,0,1,4,1
21,21,1,A0Y1V2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62872,0,1,4,1
22,22,1,A0Y1V3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62873,1,1,5,0
23,23,1,A0Y1V4.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62874,1,1,6,0
24,24,1,A0Y1V5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,62875,1,1,7,1


## Get Chapter Names for all the PDFs with outlines

In [39]:
# Functtion that runs recursively to extract the outlines of the PDF files
def nested_is_instance_check(s, level, file_name, path, chunk_no,
                             PDF_names, PDF_locations, Chunk_Nos, Chapter_Numbers, 
                             Chapter_Keys, Chapter_levels, PDF_Found_Last, Page_Found_Last):
    level +=1
    if isinstance(s, (int, list, float, complex)):
        for x in s:
            nested_is_instance_check(x, level, file_name, path, chunk_no,
                                     PDF_names, PDF_locations, Chunk_Nos, Chapter_Numbers, 
                                     Chapter_Keys, Chapter_levels, PDF_Found_Last, Page_Found_Last)
    elif level < 5:
        PDF_names.extend([file_name])
        full_path =  path + file_name
        PDF_locations.extend([full_path])
        Chunk_Nos.extend([chunk_no])
        
        chapter_title = s.title
        Chapter_Numbers.extend([0])
        Chapter_Keys.extend([chapter_title])
        
        Chapter_levels.extend([level])
        #it was observed that for a lot of PDF files we could not find the page number 
        # Hence, -999 is the error code added for those cases 
        page_error_flag = 0
        try:
            Page_Found_Last.extend([(pdfread.getDestinationPageNumber(s))])
        except: 
            Page_Found_Last.extend([-999])
            page_error_flag = 1
        
        
        if page_error_flag != 1:
            PDF_Found_Last.extend([file_name])
        else:
            PDF_Found_Last.extend([' '])
            
        

In [40]:
PDF_names = []
PDF_locations = []
Chunk_Nos = []
Chapter_Numbers = []
Chapter_Keys = []
Chapter_levels = []
PDF_Found_Last = [] # This is redundant for Outline method, unless used with the chuck method 
Page_Found_Last = []
path = test2 = "F:/Environmental Baseline Data/Version 3/Data/PDF/"


for i in range(len(df_outlines)):
#     First file in the chunk has Outline 
#     print("PDF name {}".format(df_outlines.iloc[i]['file_names']))
    level = 0
    if df_outlines.iloc[i]['outline_present'] == 1:
        pdfread = p2.PdfFileReader(df_outlines.iloc[i]['file_paths'])
        s = pdfread.outlines
        nested_is_instance_check(s, 
                                level, 
                                df_outlines.iloc[i]['file_names'], path,
                                df_outlines.iloc[i]['chunk_nos'], 
                                PDF_names, PDF_locations, Chunk_Nos, Chapter_Numbers, 
                                Chapter_Keys, Chapter_levels, PDF_Found_Last, Page_Found_Last)
        #file_name_containing_index = current_df_outlines.iloc[i]['file_names']
            
    

In [41]:
len(PDF_names)
len(PDF_locations)
len(Chunk_Nos)
len(Chapter_Numbers)
len(Chapter_Keys)
len(Chapter_levels)
len(PDF_Found_Last)
len(Page_Found_Last)
len(Chapter_Keys)

12835

12835

12835

12835

12835

12835

12835

12835

12835

In [48]:
allChaptersDF = pd.DataFrame({'PDF_names' : PDF_names, 
                              'PDF_locations' : PDF_locations,
                              'Chunk_Nos' : Chunk_Nos,
                              'Chapter_Numbers' : Chapter_Numbers,  
                              'Chapter_Keys' : Chapter_Keys,
                              'Chapter_levels' : Chapter_levels,
                              'PDF_Found_Last': PDF_Found_Last,
                              'Page_Found_Last' : Page_Found_Last})
allChaptersDF.head(5)

,PDF_names,PDF_locations,Chunk_Nos,Chapter_Numbers,Chapter_Keys,Chapter_levels,PDF_Found_Last,Page_Found_Last
0,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Environmental Impact Assessment and Mitigation...,2,A0H8C0.pdf,24
1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Introduction,3,A0H8C0.pdf,24
2,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Project Background,4,A0H8C0.pdf,24
3,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Regulatory Setting,4,A0H8C0.pdf,24
4,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Scope of Environmental Assessment,4,A0H8C0.pdf,26


In [83]:
PositiveChaptersDF = allChaptersDF[allChaptersDF['Page_Found_Last'] > -1]
PositiveChaptersDF = allChaptersDF[allChaptersDF['Chapter_levels'] < 4 ]
# len(PositiveChaptersDF)
# PositiveChaptersDF.head(5)

#PositiveChaptersDF = PositiveChaptersDF[allChaptersDF.Chunk_Nos.isin([274, 275, 276, 277])]
PositiveChaptersDF


PositiveChaptersDF.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Index - Chapter Outlines for NGTL 2021.csv")


#PositiveChaptersDF = allChaptersDF[allChaptersDF['PDF_locations'] == "F:/Environmental Baseline Data/Version 3/Data/PDF/A3E2X9.pdf"]

,PDF_names,PDF_locations,Chunk_Nos,Chapter_Numbers,Chapter_Keys,Chapter_levels,PDF_Found_Last,Page_Found_Last
0,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Environmental Impact Assessment and Mitigation...,2,A0H8C0.pdf,24
1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Introduction,3,A0H8C0.pdf,24
5,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Project Description,3,A0H8C0.pdf,27
11,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Assessment Methods,3,A0H8C0.pdf,36
20,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Air,3,A0H8C0.pdf,42
...,...,...,...,...,...,...,...,...
12828,A6T2W6.pdf,F:/Environmental Baseline Data/Version 3/Data/...,286,0,Potential Effects of Decommissioning and Aband...,3,A6T2W6.pdf,204
12829,A6T2W6.pdf,F:/Environmental Baseline Data/Version 3/Data/...,286,0,Mitigation and Reclamation,3,A6T2W6.pdf,205
12830,A6T2W6.pdf,F:/Environmental Baseline Data/Version 3/Data/...,286,0,Summary of Significance Conclusions for Potent...,3,A6T2W6.pdf,206
12833,A6T2W6.pdf,F:/Environmental Baseline Data/Version 3/Data/...,286,0,References,3,A6T2W6.pdf,207


In [84]:
len(PositiveChaptersDF)

8122

# 1. Extracting Text from the Chapter 

In [85]:
def remove_digits(chapter_key):
    """ A lot of thimes the chapter name in the index have a number at the start of the string, 
    however, when we are searching for that text in the PDF file the number and the text are seperated by /n
    Example: 
    Chapter Key in Outline : " 5.0 Environment Assessment  ""
    Chapter Key in actual PDF: "5.0
                            Environment Assessment"
    
    
    
    This function removes the digits if the and strips the text to improve the searcheability.
    New Chapter Key in Outline : "Environment Assessment"  
    """
    index = 0
    chapter_key = str(chapter_key)
    chapter_key = chapter_key.strip()
    if ' ' in chapter_key and chapter_key[0].isdigit():
        index = chapter_key.index(' ')
        chapter_key = chapter_key[index+1:]
    return(chapter_key.strip())
        
    

In [86]:
a = "5.2.4 Hardisty Terminal"
a = PositiveChaptersDF.iloc[69]['Chapter_Keys']
print(a)
print(type(a))
a = str(a)
print(' ' in a)


a = remove_digits(a)
a

TABLE 3.2  CONSULTATION ACTIVITIES WITH PROVINCIAL AGENCIES 
<class 'PyPDF2.generic.TextStringObject'>
True


'TABLE\xa03.2  CONSULTATION ACTIVITIES WITH PROVINCIAL AGENCIES'

In [87]:
def read_chapter_text(PDF_location, Start_Text, Start_Page, End_Page, End_Text,i):
    
    if Start_Page < 0:
        Chapter = "Invalid Page Number"
        return(Chapter)
    
    
    print("Reading PDF {} and the start text {} at i = {}".format(PDF_location, Start_Text, i))
    object = p2.PdfFileReader(PDF_location)
    
    Start_Text = remove_digits(Start_Text)
    End_Text = remove_digits(End_Text)
        
    # get number of pages
    NumPages = object.getNumPages()
    if NumPages < End_Page:
        End_Page = NumPages - 1
        
    
    
    Chapter = ""    
    for i in range(Start_Page, End_Page+1):
        PageObj = object.getPage(i)
        
#         print("*****************************************")
#         print("*****************************************")
#         print("*****************************************")
        
        try:
            Text = PageObj.extractText()
            Text = Text.replace('\n','').replace('\r,'')
            #print("The Text Extracted for page no {} is {}".format(i, Text))
        except:
            Text = "Text could not be extracted for File {} for Page Number {}".format(PDF_location, i)
            print(Text)
            continue
        
        if Start_Page == End_Page+1:
            try:
                Text = Text.split(Start_Text)[1]
            except:
                 print("COULD NOT FIND {} IN THE PAGE WITH THE TEXT".format(Start_Text))
            try:
                Text = Text.split(End_Text)[0]
            except:#                 
                print("COULD NOT FIND {} IN THE PAGE WITH THE TEXT".format(End_Text))       

            Chapter = Start_Text + Text
            
        
        elif i == Start_Page:
            try:
                Text = Text.split(Start_Text)[1]
            except:
                print("Never Mind: COULD NOT FIND {} IN THE PAGE WITH THE TEXT".format(Start_Text))
            Chapter = Start_Text + Text
            
        elif i == End_Page + 1:
            try:
                Text = Text.split(End_Text)[0]
            except:
                 print("Never Mind: COULD NOT FIND {} IN THE PAGE WITH THE TEXT".format(End_Text))
            Chapter = Chapter + Text
                
        else:
            Chapter = Chapter + Text   
    return(Chapter)

In [88]:
PositiveChaptersDF = PositiveChaptersDF
Chapter_Texts = []
for i in range(len(PositiveChaptersDF)):
    try:
        if PositiveChaptersDF.iloc[i+1]['PDF_Found_Last'] == PositiveChaptersDF.iloc[i]['PDF_Found_Last']:
            End_Page = PositiveChaptersDF.iloc[i+1]['Page_Found_Last']
            End_Text = PositiveChaptersDF.iloc[i+1]['Chapter_Keys']
        else:
            # For cases when the chapter in the last chapter in the outline for that 
            End_Page = 99999
            End_Text = "Vibudh loves chocolates." # Random Text which is not there in the PDF 
        
    except:
        # For case when the chapter in the last chapter in the DataFrame  
        End_Page = 99999
        End_Text = "Vibudh loves chocolates." # Random Text which is not there in the PDF 
        
    Chapter_Text = read_chapter_text(PositiveChaptersDF.iloc[i]['PDF_locations'], 
                      PositiveChaptersDF.iloc[i]['Chapter_Keys'],
                      PositiveChaptersDF.iloc[i]['Page_Found_Last'],
                      End_Page, End_Text,i)
                      
    Chapter_Texts.extend([Chapter_Text])
        
len(Chapter_Texts)
len(PositiveChaptersDF)

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0H8C0.pdf and the start text Environmental Impact Assessment and Mitigation Plan at i = 0


Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0H8C0.pdf and the start text Introduction at i = 1
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0H8C0.pdf and the start text Project Description at i = 2
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0H8C0.pdf and the start text Assessment Methods at i = 3
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0H8C0.pdf and the start text Air at i = 4
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0H8C0.pdf and the start text Terrain and Soils at i = 5
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0H8C0.pdf and the start text Vegetation at i = 6
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0H8C1.pdf and the start text Wildlife and Wildlife Habitat at i = 21
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Y1T7.pdf and the start text EXECUTIVE SUMMARY  at i = 28
Reading PDF F:/Environmental Baseline Data/Version 3/Dat

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Y1U8.pdf and the start text  TABLE 6.13  SIGNIFICANCE EVALUATION OF ADVERSE RESIDUAL EFFECTS OF IDENTIFIED BIOPHYSICAL AND SOCIO-ECONOMIC ELEMENTS OF LINE 13 REVERSAL  at i = 100
Never Mind: COULD NOT FIND TABLE 6.13  SIGNIFICANCE EVALUATION OF ADVERSE RESIDUAL EFFECTS OF IDENTIFIED BIOPHYSICAL AND SOCIO-ECONOMIC ELEMENTS OF LINE 13 REVERSAL IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Y1U9.pdf and the start text 7.0 CUMULATIVE EFFECTS ASSESSMENT  at i = 101
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Y1U9.pdf and the start text 7.1 Methodology  at i = 102
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Y1U9.pdf and the start text  Figure 7.1 Location of Known Proposed Oil and Gas Developments within the Vicinity of the Southern Lights Project  at i = 103
Never Mind: COULD NOT FIND Figure 7.1 Location of Known Proposed Oil and Gas Developments

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Y6.pdf and the start text TABLE 1.2  SUMMARY OF RECENT SUPPORTING STUDIES ALONG THE PROPOSED PIPELINE ROUTE at i = 193
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Y6.pdf and the start text TABLE 1.3  SUMMARY OF RECENT PIPELINE APPLICATIONS AND ASSOCIATED SUPPORTING STUDIES ALONG THE PROPOSED ALBERTA CLIPPER PIPELINE ROUTE at i = 194
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Y6.pdf and the start text TABLE 1.4  SUMMARY OF RECENT ENBRIDGE STATION APPLICATIONS AND ASSOCIATED SUPPORTING STUDIES AT STATIONS/TERMINALS WITH PROPOSED ADDITIONS at i = 195
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Y6.pdf and the start text 1.5  Purpose of the Document  at i = 196
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Y6.pdf and the start text 1.6 Project Team  at i = 197
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Y6.pdf and the 

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Z0.pdf and the start text TABLE 5.2  PHYSIOGRAPHIC REGIONS TRAVERSED BY THE PROPOSED PIPELINE ROUTE IN SASKATCHEWAN  at i = 268
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Z0.pdf and the start text TABLE 5.3  PHYSIOGRAPHIC REGIONS TRAVERSED BY THE PROPOSED PIPELINE ROUTE IN MANITOBA at i = 269
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Z0.pdf and the start text TABLE 5.4  GEOLOGIC FORMATIONS UNDERLYING THE PROPOSED PIPELINE ROUTE IN ALBERTA at i = 270
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Z0.pdf and the start text TABLE 5.5  GEOLOGIC FORMATIONS UNDERLYING THE PROPOSED PIPELINE ROUTE IN SASKATCHEWAN at i = 271
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A0Z0Z0.pdf and the start text  TABLE 5.6  GEOLOGIC FORMATIONS UNDERLYING THE PROPOSED PIPELINE ROUTE IN MANITOBA at i = 272
Reading PDF F:/Environmental Baseline Data/Version 3/Data/

...

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2G5.pdf and the start text 10.6 ASSESSMENT OF CUMULATIVE EFFECTS ON HUMAN OCCUPANCY AND RESOURCE USE at i = 7899
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2G5.pdf and the start text 10.7 DETERMINATION OF SIGNIFICANCE AND PREDICTION CONFIDENCE at i = 7900
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2G5.pdf and the start text 10.8 FOLLOW-UP AND MONITORING at i = 7901
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2G5.pdf and the start text 11.0 ASSESSMENT OF EFFECTS ON TRADITIONAL LAND AND RESOURCE USE at i = 7902
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2G5.pdf and the start text 11.1 SCOPE OF ASSESSMENT at i = 7903
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2G5.pdf and the start text 11.2 EXISTING CONDITIONS FOR TRADITIONAL LAND AND RESOURCE USE at i = 7904
Reading PDF F:/Environmental Baseline Data/Version 3/Data/P

Never Mind: COULD NOT FIND Appendix I Traditional Knowledge Technical Data Report IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2H6.pdf and the start text Table of Contents at i = 7995
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2H6.pdf and the start text Abbreviations at i = 7996
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2H6.pdf and the start text 1.0 INTRODUCTION  at i = 7997
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2H6.pdf and the start text 2.0 DEFINITIONS at i = 7998
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2H6.pdf and the start text 3.0 METHODS at i = 7999
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2H6.pdf and the start text 4.0 ASSESSMENT AREAS at i = 8000
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2H6.pdf and the start text 5.0 PROJECT SETTING at i = 8001
Reading PDF F:/Environmental Baseline Data/Ve

Never Mind: COULD NOT FIND Appendix 1A Environmental Protection Plan for the Proposed North Corridor Expansion – North Central Corridor Loop (North Star Section 2) IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2V9.pdf and the start text Appendix 1B Environmental Protection Plan for the Proposed North Corridor Expansion – North Central Corridor Loop (Red Earth Section 3) at i = 8086
Never Mind: COULD NOT FIND Appendix 1B Environmental Protection Plan for the Proposed North Corridor Expansion – North Central Corridor Loop (Red Earth Section 3) IN THE PAGE WITH THE TEXT
Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2W2.pdf and the start text Appendix 1C Environmental Protection Plan for the Proposed North Corridor Expansion – Northwest Mainline Loop No. 2 (Bear Canyon North Extension) at i = 8087
Never Mind: COULD NOT FIND Appendix 1C Environmental Protection Plan for the Proposed North Corridor Expansion – Northwest Mainline

8122

8122

In [89]:
len(Chapter_Texts)
len(PositiveChaptersDF)

8122

8122

In [90]:
PositiveChaptersDF['Chapter Text'] = Chapter_Texts
PositiveChaptersDF.head(5)

PositiveChaptersDF.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Index - Chapter Texts for NGTL 2021.csv")

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:1]


,PDF_names,PDF_locations,Chunk_Nos,Chapter_Numbers,Chapter_Keys,Chapter_levels,PDF_Found_Last,Page_Found_Last,Chapter Text
0,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Environmental Impact Assessment and Mitigation...,2,A0H8C0.pdf,24,Environmental Impact Assessment and Mitigation...
1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Introduction,3,A0H8C0.pdf,24,Introduction 13.1.1 Project Background EnCana ...
5,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Project Description,3,A0H8C0.pdf,27,Project Description 13.2.1 Scope of Project Th...
11,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Assessment Methods,3,A0H8C0.pdf,36,Assessment Methods 13.3.1 Issues Scoping Issue...
20,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1,0,Air,3,A0H8C0.pdf,42,Air 13.4.1 Baseline Setting The Project area i...


In [91]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('F:/Environmental Baseline Data/Version 3/Indices/Index - Chapter Texts for NGTL 2021.xlsx', engine='xlsxwriter')
PositiveChaptersJSON.to_excel(writer, sheet_name='Sheet1')

AttributeError: 'str' object has no attribute 'to_excel'

In [ ]:
PositiveChaptersJSON = PositiveChaptersDF.to_json(orient = 'index')
PositiveChaptersJSON

with open('F:/Environmental Baseline Data/Version 3/Indices/Index - Chapter Texts for NGTL 2021.txt', 'w') as outfile:
    json.dump(PositiveChaptersJSON, outfile)

# 2.1 Spacy location - PDF Location/Date Identifier

This was better done by the students of UBC using the HTML files. 

# Rough Work

In [189]:
Echap = 0
for i in Chapter_Texts: 
    if i == "":
        Echap = Echap + 1
        print("Empty Chap")
        
Echap

0

In [118]:
a = ""
b = "vob"
a = a +b 
a

Start_Text = "SA"
Text = "Vibudh SAingh"
Text = Text.split(Start_Text)[1]
Text + b 
PositiveChaptersDF.iloc[8000]['Chapter_Keys']

'vob'

'inghvob'

'4.0 ASSESSMENT AREAS'

In [125]:
for i in range(8000, len(PositiveChaptersDF)):
    try:
        if PositiveChaptersDF.iloc[i+1]['PDF_Found_Last'] == PositiveChaptersDF.iloc[i]['PDF_Found_Last']:
            End_Page = PositiveChaptersDF.iloc[i+1]['Page_Found_Last']
            End_Text = PositiveChaptersDF.iloc[i+1]['Chapter_Keys']
        else:
            # For cases when the chapter in the last chapter in the outline for that 
            End_Page = 99999
            End_Text = "Vibudh Rocks" # Random Text which is not there in the PDF 
        
    except:
        # For case when the chapter in the last chapter in the DataFrame  
        End_Page = 99999
        End_Text = "Vibudh Rocks" # Random Text which is not there in the PDF 
        
    Chapter_Text = read_chapter_text(PositiveChaptersDF.iloc[i]['PDF_locations'], 
                      PositiveChaptersDF.iloc[i]['Chapter_Keys'],
                      PositiveChaptersDF.iloc[i]['Page_Found_Last'],
                      End_Page, End_Text)
                      
    Chapter_Texts.extend([Chapter_Text])
        
len(Chapter_Texts)
len(PositiveChaptersDF)

Reading PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6T2W4.pdf
*****************************************
*****************************************
*****************************************
The Text Extracted for page no 460 is FINAL
 REPORT
 rwdi.com
 This 
document is intended for the sole use of the party to whom it is addressed and may contain information that is privileged and
/or confidential. If you 
have received this in error, please notify us immediately. ® RWDI name and logo are registered trademarks 
in Canada and the United States of America
  NORTH CORRIDOR 
EXPANSION PROJECT
  NOISE IMPACT ASSESSM
ENT
 RWDI #
1803694
 April 1, 2019
   SUBMITTED TO
 Barbara Taylor
 TransCanada
 450 1st St SW
 Calgary, Alberta
 T2P 5H1
 barbara_taylor
@transcanada.com
    SUBMITTED BY
 Candace Bell, M.Sc.
 Project Manager
 Candace.Bell
@rwdi.com
     RWDI
 Suite 1000, 736
-8th Avenue S.W.
 Calgary, Alberta, Canada
 T2P 1H4
 T: 
403.232.6771
 F: 
519.823.1316
    
Never Mind
******

The Text Extracted for page no 466 is NOISE IMPACT ASSESSM
ENT
 NORTH CORRIDOR EXPAN
SION PROJECT
 RWDI#
1803694
 April 1, 2019
  rwdi.com
   
Page 
4 
     ﬁdaytimeﬂ adjustment is used for the hours between 07:00 and 22:00. The daytime PSL would include a +10 dBA 
adjustment to the nighttime PSL as defined Directive 038.
 Where no noise sensitive receptors have been identified, the 4
0 dBA nighttime L
EQ limit, 
at the 1.5 km criteria 
boundary defined in 
AER 
Directive 038 will be used.
 As stated in AER 
Directive 038
, 
the natural ambient sound level (ASL) is generally defined as 5 dBA less than the 
BSL
. Adjustments for the presence of 
non
-p
roject related ambient sound levels are defined in 
AER Directive 038
. These
 adjustments are applied as such situations are identified in the study 
based on
 the baseline noise 
measurements
 and expected operating conditions
.  With the cumulative approach that is required, compliance of predicted 
project sound levels with the PSL cr

The Text Extracted for page no 469 is NOISE IMPACT ASSESSM
ENT
 NORTH CORRIDOR EXPAN
SION PROJECT
 RWDI#
1803694
 April 1, 2019
  rwdi.com
   
Page 
7 
     4 FACILITY SOUND SOURCES
 4.1
 Adjacent
 Facilities
 To identify facilities that may contribute to the acoustic environment
 near 
the 
Hidden Lake
 North 
Unit 
Addition
, publicly available databases were reviewed to determine the locations of potential third
-party 
energy
-related 
contributors. The following databases were consulted to determine the locations of facilities in the vicinity of the 
Hidden Lake North 
Unit Addition
: 
 Alberta Energy Regulator ST37 
Œ 
Alberta well listing (AER
 2018
a)  Alberta Energy Regulator ST102 
Œ 
Alberta facility listing (AER
 2018
b)
 The f
acilities shown in Table 2
 were further 
verified 
by a field survey and operation was confirmed by follow
-up 
with CNRL.  The 
Hidden Lake North Unit Addition 
and existing facilities are shown in 
Figure 
1. 
Table 
2: Adjacent
 Facilities Consid

The Text Extracted for page no 474 is NOISE IMPACT ASSESSM
ENT
 NORTH CORRIDOR EXPAN
SION PROJECT
 RWDI#
1803694
 April 1, 2019
  rwdi.com
            
Page 
12      Item
 Source 
Type
1 QTY
 Levels at Octave Band Center Frequencies (dB)
 Overall Sound Power
 Reference 
Source
 31.5
 63
 125
 250
 500
 1,000
 2,000
 4,000
 8,000
 (dBA)
 (dB)
 Bld Louver Set 3
 P 
1 
79.7
 79.6
 83.5
 83.6
 87.3
 87.0
 93.3
 90.9
 85.8
 97.4
 97.3
 (3)(4)(5)
 Lube Oil Cooler
 P 
1 
104.5
 111.5
 108.5
 101.5
 96.5
 93.5
 89.5
 85.5
 80.5
 100.0
 114.2
 (3)(4)(5)
 Bld Roof
 A 
1 
99.6
 98.6
 100.4
 96.6
 91.2
 88.0
 92.2
 85.9
 78.8
 96.9
 105.5
 (3)(4)(5)
 Bld Wall
 V 
2 
96.6
 95.6
 97.4
 93.6
 88.2
 85.0
 89.2
 82.9
 75.8
 93.9
 102.6
 (3)(4)(5)
 Bld Wall
 V 
2 
97.6
 96.6
 98.4
 94.6
 89.2
 86.0
 90.2
 83.9
 76.8
 94.9
 103.5
 (3)(4)(5)
 Hidden Lake Compressor Station 
- 
Control Building (Boiler Running Only, Genset Off)
 Boiler Exhaust
 P 
1 
92.1
 91.1
 88.1
 86.1
 74.1
 67.1
 59.1
 56.1
 42.1
 79

The Text Extracted for page no 481 is 































*****************************************
*****************************************
*****************************************
The Text Extracted for page no 482 is NOISE IMPACT ASSESSM
ENT
 NORTH CORRIDOR EXPAN
SION PROJECT
 RWDI#
1803694
 April 1, 2019
  rwdi.com
 Page 
20   5.4 Low Frequency Noise
 The C
-weighted sound level (dBC) results have been reviewed to determine if there is a potential for LFN at the 
boundary receptor. The differences between the 
Hidden Lake North Unit Addition
-only C
-weighted and 
A-
weighted values at the chosen boundary receptor is shown in 
Table 
8. 
Table 
8: Low Frequency Noise Assessment 
 Receptor ID
 C-Weighted 
Sound Level
 (dBC)
 A-Weighted 
Sound Level
 (dBA)
 dBC
-dBA
 Potential LFN 
Condition?
 (Yes/No)
 BR1
 60.6
 36.1
 24.5
 Yes
  The 
calculation for LFN shows l
evels above the AER threshold.
 However, 
because
 the 
facility is in a remote 
location with no dwelling


...

*****************************************
*****************************************
*****************************************
The Text Extracted for page no 206 is  Appendix 12. Preliminary 
Decommissioning or Abandonment
 Assessment
  A12
-6 AX0917181256CGY
 abandonment will be in accordance with the most current body of scientific knowledge and NGTL™s 
established and proven mitigation measures and construct
ion practices available at the time of 
decommissioning or abandonment. 
 Land disturbance activities related to the removal of pipeline or associated facilities would be reclaimed to 
the equivalent land use at that time. The methods for evaluating the succe
ss of reclamation would be 

based on the principle that the success of land reclamation is measured against the adjacent site 
conditions, while taking into consideration the status of reclamation at the time of the assessment. 
 The reclamation objectives or
 principles to be applied to abandonment of the Project will also

The Text Extracted for page no 204 is  Appendix 12. Preliminary 
Decommissioning or Abandonment
 Assessment
  A12
-4 AX0917181256CGY
 Table 
A12
-1. 
Potential 
Element Interaction
s with the North Corridor Expansion Project
 Biophysical and 
Socio
-Economic 
Element
 Interaction (Yes / No)
 Description of Potential 
Interactions During 
Decommissioning and Abandonment Activities
 Pipeline
 (North 
Star Section 2,
 Red Earth 
Section 3,
 and
 Bear Canyon 
North 
Extension
) Facilities 
(including 
Hidden Lake 
North 
Unit 
Addition)
 Heritage Resources
 No No No heritage resources are anticipated on the previously 
disturbed 
PCF
. Traditional Land and 
Resource Use
 Yes
 Yes
 Decommissioning and abandonment activities could alter or 
temporarily disrupt use of trails and travelways,
 and subsistence activities.
 Social and Cultural 
Well
-being
 Yes
 Yes
 Decommissioning and abandonment activities could 
temporarily affect the local population due to the influx of 
temporary workforce

The Text Extracted for page no 206 is  Appendix 12. Preliminary 
Decommissioning or Abandonment
 Assessment
  A12
-6 AX0917181256CGY
 abandonment will be in accordance with the most current body of scientific knowledge and NGTL™s 
established and proven mitigation measures and construct
ion practices available at the time of 
decommissioning or abandonment. 
 Land disturbance activities related to the removal of pipeline or associated facilities would be reclaimed to 
the equivalent land use at that time. The methods for evaluating the succe
ss of reclamation would be 

based on the principle that the success of land reclamation is measured against the adjacent site 
conditions, while taking into consideration the status of reclamation at the time of the assessment. 
 The reclamation objectives or
 principles to be applied to abandonment of the Project will also be in 
accordance with legislative and regulatory requirements in place at that time. The primary goal of 
reclamation follow

210

8122

In [95]:
PCDF = PositiveChaptersDF[8100:]
PCDF
len(PCDF)

,PDF_names,PDF_locations,Chunk_Nos,Chapter_Numbers,Chapter_Keys,Chapter_levels,PDF_Found_Last,Page_Found_Last
12742,A6T2W4.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,NOISE IMPACT ASSESSMENT,3,A6T2W4.pdf,460
12752,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Appendix 6 Aquatics Technical Data Report,2,A6T2W5.pdf,0
12753,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Aquatics Technical Data Report for the Propose...,3,A6T2W5.pdf,1
12763,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Appendix 7 Wetland Technical Data Report,2,A6T2W5.pdf,59
12764,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Wetland Technical Data Report for the Proposed...,3,A6T2W5.pdf,60
12772,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Appendix 8 Vegetation Technical Data Report,2,A6T2W5.pdf,82
12773,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Vegetation Technical Data Report for the Propo...,3,A6T2W5.pdf,83
12783,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Appendix 9 Wildlife Technical Data Report,2,A6T2W5.pdf,155
12784,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,285,0,Wildlife Technical Data Report for the Propose...,3,A6T2W5.pdf,156
12795,A6T2W6.pdf,F:/Environmental Baseline Data/Version 3/Data/...,286,0,Appendix 10 Caribou Habitat Restoration and Of...,2,A6T2W6.pdf,0


22

In [ ]:
object = p2.PdfFileReader(PDF_location)
        
    # get number of pages
    NumPages = object.getNumPages()
    if NumPages < End_Page:
        End_Page = NumPages - 1
    
    Chapter = ""    
    for i in range(Start_Page, End_Page+1):
        PageObj = object.getPage(i)
        
        try:
            if i == Start_Page:
                Text = PageObj.extractText()
                try:
                    Text = Text.split(Start_Text)[1]
                except:
                    print("Never Mind")
                Chapter.append(Start_Text + Text)

In [86]:
PDF_location = PositiveChaptersDF.iloc[1]['PDF_locations']
object = p2.PdfFileReader(PDF_location)
# get number of pages
NumPages = object.getNumPages()

NumPages

PageObj = object.getPage(27)
Text = PageObj.extractText()
Text

107

' ENVIRONMENTAL \nIMPACT \nASSESSMENTAND \nMITIGATION \nPLAN\n Page 13-4\n   EnCana Ekwan Pipeline Inc. \n13.2 Project Description \n13.2.1 Scope of Project \nThis Project will involve the construction and oper\nation of approximately \n83 km of 61 cm \n(51 miles of 24 inch) pipeline. \nNo upstream modifications to \nthe EnCana Sierra Gas Plant \nor downstream modifications to the NGTL \nNorthwest Mainline pipeline system will be \nrequired to accommodate the new gas flow \nthrough the proposed Pipeline. Therefore, based on the ‚principal project/a\nccessory test™ provided by the \nCanadian Environmental \nAssessment Act \nTraining Compendium (Canadian Envi\nronmental Assessment Agency \n1994), the ‚scope of the project™ \nfrom the perspective of the \nCanadian Environmental Assessment Act \ncan be defined as the physical works \nassociated with the construction and \noperation of the Pipeline only.  Project details are provided in Volume 1 of\n the Application document. The informatio

In [ ]:
Chapter_Texts = []
for i in range(8100, len(PositiveChaptersDF)):
    try:
        if PositiveChaptersDF.iloc[i+1]['PDF_Found_Last'] == PositiveChaptersDF.iloc[i]['PDF_Found_Last']:
            End_Page = PositiveChaptersDF.iloc[i+1]['Page_Found_Last']
            End_Text = PositiveChaptersDF.iloc[i+1]['Chapter_Keys']
        else:
            # For cases when the chapter in the last chapter in the outline for that 
            End_Page = 99999
            End_Text = "Vibudh Rocks" # Random Text which is not there in the PDF 
        
    except:
        # For case when the chapter in the last chapter in the DataFrame  
        End_Page = 99999
        End_Text = "Vibudh Rocks" # Random Text which is not there in the PDF 
        
    Chapter_Text = read_chapter_text(PositiveChaptersDF.iloc[i]['PDF_locations'], 
                      PositiveChaptersDF.iloc[i]['Chapter_Keys'],
                      PositiveChaptersDF.iloc[i]['Page_Found_Last'],
                      End_Page, End_Text)
                      
    Chapter_Texts.extend([Chapter_Text])
        
len(Chapter_Texts)
len(PositiveChaptersDF)

In [ ]:
['Username', ' How are you today?']

In [65]:
s1 = "Vibudh Rocks. He is awesome"
s1.split('%%%%')[0]
s1.split('He')[1]

'Vibudh Rocks. He is awesome'

' is awesome'

In [70]:
for i in range(5,5):
    print("Viobud")

In [ ]:
# Variables for Initialization of the data frame for Index 4 - Chapters.csv
file_names = []
found_file_names = []
chapters = []
levels = []
pages = []
all_df_chapters = pd.DataFrame({'file_names': file_names, 'found_file_names': found_file_names, 'chapters': chapters, 'pages' : pages, 'levels' : levels})

for current_chunk in range(10,12):#chunk+1):
    current_df_outlines = df_outlines[df_outlines.chunk_nos == current_chunk]
    
    chunk_has_outline = 0
    chunk_noOutline_hasTOC = 0
    
    file_name_containing_index = '0'
    
    p_file_names = []
    p_found_file_names = []
    p_chapters = []
    p_levels = []
    p_pages = []
    level = 0
    # Going through all the PDFs - Chunk by Chunk
    for i in range(len(current_df_outlines)):
        # First file in the chunk has Outline 
        if current_df_outlines.iloc[i]['outline_present'] == 1:
            chunk_has_outline = 1
            print("stage 1")
            pdfread = p2.PdfFileReader(current_df_outlines.iloc[i]['file_paths'])
            s = pdfread.outlines
            
            nested_is_instance_check(s, 
                                     level, 
                                     current_df_outlines.iloc[i]['file_names'], 
                                     p_file_names, p_found_file_names, 
                                     p_chapters, p_levels, p_pages)
            file_name_containing_index = current_df_outlines.iloc[i]['file_names']
            continue
            
        if count_occorances(p_pages, -999) > 1 and chunk_has_outline == 1:
            print("stage 2: file is {}".format(current_df_outlines.iloc[i]['file_names']))
            start_flag = 0
            page_no = -999
            #print(p_pages)
            for j in range(len(p_chapters)):
                if p_pages[j] >= 0 and p_levels[j] < 5: # probably could have also used p_pages[j] >= - 998
                    print("start flag is on")
                    start_flag = 1
                    continue
                
                if start_flag != 1:
                    continue
                    
                if start_flag == 1 and p_pages[j] <= 0 and p_levels[j] < 5:
                    print("stage 3: finding page no for {} with page{}".format(p_chapters[j], p_pages[j]))
                    page_no, found_file_name = find_page_no(current_df_outlines.iloc[i]['file_paths'], 
                                                            p_chapters[j])
                    p_pages[j] = page_no
                    p_found_file_names = current_df_outlines.iloc[i]['file_names']
                    
                if page_no == -999 and p_levels[j] < 5:
                    print("Not found")
                    break # if the first keyword is not found then dont look any further
                else:
                    print("New Found ")
            
    df_chapters = pd.DataFrame({'file_names': p_file_names, 'found_file_names': p_found_file_names, 'chapters': p_chapters, 'pages' : p_pages, 'levels' : p_levels})
    all_df_chapters = pd.concat([all_df_chapters, df_chapters])
        
    
    

In [ ]:
previous_pdf = 0
#pdf_name = []
mean_area = {}
page_numbers = {}
sum_area = 0
count_pdf = 0

# Get Mean arae for each PDF files 
for i in range(len(Xdf_dataset)):
    current_pdf, page_no = Xdf_dataset.iloc[i].dataID_pageNo.split("_")
    
    if previous_pdf != current_pdf:
        
        if previous_pdf != 0:
            mean_area[previous_pdf] = sum_area/count_pdf
            page_numbers[previous_pdf] = count_pdf
            sum_area = 0
            count_pdf = 0
        previous_pdf = current_pdf
    
    sum_area = sum_area + Xdf_dataset.iloc[i].Area_of_images
    count_pdf =count_pdf +1
    
mean_area[previous_pdf] = sum_area/count_pdf
page_numbers[previous_pdf] = count_pdf

len(mean_area)
len(page_numbers)
#len(pdf_name)
len(DataIDHand)
mean_area

area_ratio = []
pdf_page_ratio = []
type(area_ratio)
for i in range(len(Xdf_dataset)):
    current_pdf, page_no = Xdf_dataset.iloc[i].dataID_pageNo.split("_")
    area_ratio.append(Xdf_dataset.iloc[i].Area_of_images/mean_area[current_pdf])
    pdf_page_ratio.append(int(page_no)/page_numbers[current_pdf])
len(area_ratio)
#len(page_numbers)
Xdf_dataset['area_ratio'] = area_ratio
Xdf_dataset['pdf_page_ratio'] = pdf_page_ratio
Xdf_dataset.head()